In [12]:

import requests

res = requests.get("https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch")

from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, 'html.parser')

all_laptops = soup.find_all('div', attrs= {'class': 'right-block'})


with open('sigma_laptops.csv', mode = 'w',  encoding='utf-8') as fd:
    fd.write('brand, model, cpu, gpu, ram, resolution_type, resolution, link, warranty, new_price, old_price, availability\n')
    
    for index in range(10): #range(len(all_laptops))
        processor = "Unknown"
        gpu = "Unknown"
        resolution_type = "Unknown"
        screen_resolution = "Unknown"
        ram = "Unknown"
        print(f'Scraping laptop {index}...')

        new_price = all_laptops[index].find('p').find('span', {'class': "price-new"}).text.strip()

        old_price = all_laptops[index].find('p').find('span', {'class': "price-old"}).text.strip()

        try:
            availability = all_laptops[index].find('div', {'class' : 'row'}).find('span', {'class': 'stock'}).text.strip()
        except:
            availability = all_laptops[index].find('div', {'class' : 'row'}).find('span', {'class': 'stock_N'}).text.strip()

        if float(old_price) > 10000:

            link1 = 'https://www.sigma-computer.com/' +  all_laptops[index].find('a').attrs['href']

            model_link = requests.get(link1)

            model_soup = BeautifulSoup(model_link.text, 'html.parser')

            try:

                warranty = model_soup.find('div', {'class': 'inner-box-desc'}).find('em').text

            except:

                continue


            model =  model_soup.find('div', {'class': 'model'}).find('a').text.strip()

            brand =  model_soup.find('div', {'class': 'brand'}).find('a').text.strip()


            try: 
                if 'Processor' in model_soup.find('div', {'class': 'inner-box-desc'}).find('p').text:

                    features = model_soup.find('div', {'class': 'inner-box-desc'}).find('p').get_text(separator="\n", strip=True)
                    
                    lines = features.split("\n")
                
                    specs = {}

                    keywords = ["Processor", "Graphics", "Memory", "Display", "GPU", "RAM"]

                    current_key = None
                    for line in lines:
                        line = line.strip()
                        if line in keywords:
                            current_key = line
                        elif current_key:
                            specs[current_key] = line
                            current_key = None
                    
                    processor = specs['Processor']
                    graphics = specs['Graphics']
                    display = specs['Display']
                    ram = specs['Memory']

                    import re

                    screen_resolution_pattern = r'\b(\d{3,4}\s*[xX×]\s*\d{3,4})\b'
                    screen_resolution_match = re.search(screen_resolution_pattern, display)
                    
                    if screen_resolution_match:
                        screen_resolution = screen_resolution_match.group(1)

                    else:
                        Screen_resolution=  'unknown'

                    resolution_type_match = re.search(r'\b(FHD|QHD|4K|UHD|HD|WQHD|1080p)\b', display, re.IGNORECASE)
                    resolution_type = resolution_type_match.group(1) if resolution_type_match else "Unknown"

                    processor_match = re.search(r'\b(i[3579])\b', processor)
                    processor = processor_match.group(1) if processor_match else "Unknown"

                    gpu_match = re.search(r'(UHD|RTX|GTX|MX|Vega|Iris|Xe)\b', graphics, re.IGNORECASE)
                    gpu = gpu_match.group(1) if gpu_match else "Unknown"
            
                    ram_size_match = re.search(r'(\d+GB)', ram)
                    ram = ram_size_match.group(1) if ram_size_match else "Unknown"
                
            except:
                pass
           

               
            try:        
                if 'Processor' in model_soup.find('div', {'class': 'inner-box-desc'}).text:

                    specs = model_soup.find('div', {'class': 'inner-box-desc'}).find_all('li')

                    for spec in specs:
                
                        if 'processor' in spec.text.lower():
                            processor = spec.text
                            processor_match = re.search(r'\b(i[3579])\b', processor)
                            processor = processor_match.group(1) if processor_match else "Unknown"

                        elif 'graphics:' in spec.text.lower():

                            graphics2 = spec.text
                            gpu_match1 = re.search(r'\b(UHD|RTX|GTX|MX|Vega|Iris|Xe)\b', graphics2)
                            gpu = gpu_match1.group(1) 

                        elif 'graphics card:' in spec.text.lower():

                            graphics2 = spec.text
                            gpu_match1 = re.search(r'\b(UHD|RTX|GTX|MX|Vega|Iris|Xe)\b', graphics2)
                            gpu = gpu_match1.group(1)    

                        elif 'gpu:' in spec.text.lower():

                            graphics2 = spec.text
                            gpu_match1 = re.search(r'\b(UHD|RTX|GTX|MX|Vega|Iris|Xe)\b', graphics2)
                            gpu = gpu_match1.group(1)        

                        elif 'ram:' in spec.text.lower():

                            ram = spec.text
                            ram_match = re.search(r'(\d+GB)', ram)
                            ram = ram_match.group(1) if ram_match else "Unknown"

                        
                        elif 'memory:' in spec.text.lower():

                            ram = spec.text
                            ram_match = re.search(r'(\d+GB)', ram)
                            ram = ram_match.group(1) if ram_match else "Unknown"
                 
                        elif 'display' in spec.text.lower():

                            display = spec.text
                            screen_resolution_pattern = r'\b(\d{3,4}\s*[xX×]\s*\d{3,4})\b'
                            screen_resolution_match = re.search(screen_resolution_pattern, display)
                            
                            if screen_resolution_match:
                                screen_resolution = screen_resolution_match.group(1)
        
                            else:
                                Screen_resolution =  spec.text.split(': ')[-1]

                            resolution_type_match = re.search(r'\b(FHD|QHD|4K|UHD|HD|WQHD|1080p)\b', display)
                            resolution_type = resolution_type_match.group(1) if resolution_type_match else "Unknown"
                        
        
            except:
                pass                                
            

                
            try:
                if 'Processor' in model_soup.find('div', {'class': 'inner-box-desc'}).find('h3', {'class', 'modtitle'}).text:
                    specs = model_soup.find('div', {'class': 'inner-box-desc'}).find('h3', {'class', 'modtitle'}).find_all('div')


                    for i in range(len(specs)):
                        if 'processor' in specs[i].text.lower():
                            processor = specs[i].text
                            processor_match = re.search(r'\b(i[3579])\b', processor)
                            processor = processor_match.group(1) if processor_match else "Unknown"

                        elif 'ram:' in specs[i].text.lower():

                            ram = specs[i].text
                            ram_match = re.search(r'(\d+GB)', ram)
                            ram = ram_match.group(1) if ram_match else "Unknown"
                            
                        elif 'graphics' in specs[i].text.lower():

                            graphics = specs[i].text
                            gpu_match = re.search(r'(UHD|RTX|GTX|MX|Vega|Iris|Xe)\b', graphics)
                            gpu = gpu_match.group(1) if gpu_match else specs[i].text.split(': ')[-1]

                        

                        elif 'gpu' in specs[i].text.lower():

                            graphics = specs[i].text
                            gpu_match = re.search(r'(UHD|RTX|GTX|MX|Vega)\b', graphics)
                            gpu = gpu_match.group(1) if gpu_match else "Unknown"
                    
                        elif 'display' in specs[i].text.lower():

                            display = specs[i].text
                            screen_resolution_pattern = r'\b(\d{3,4}\s*[xX×]\s*\d{3,4})\b'
                            screen_resolution_match = re.search(screen_resolution_pattern, display)
                            
                            if screen_resolution_match:
                                screen_resolution = screen_resolution_match.group(1)

                            else:
                                Screen_resolution=  'unknown'

                            resolution_type_match = re.search(r'\b(FHD|QHD|4K|UHD|HD|WQHD|1080p)\b', display, re.IGNORECASE)
                            resolution_type = resolution_type_match.group(1) if resolution_type_match else "Unknown"
                            
                        elif 'memory' in specs[i].text.lower():

                            ram = specs[i].text
                            ram_match = re.search(r'(\d+GB)', ram)
                            ram = ram_match.group(1) if ram_match else "Unknown"
                

            except:

                pass  
        else:
            continue   

        if processor == 'Unknown':
            continue
                
                
                    
        fd.write(f'\"{brand}\",\"{model}\",\"{processor}\",\"{gpu}\",\"{ram}\",\"{resolution_type}\",\"{screen_resolution}\",\"{link1}\",\"{warranty}\",\"{new_price}\",\"{old_price}\",\"{availability}\"\n')




Scraping laptop 0...
Scraping laptop 1...
Scraping laptop 2...
Scraping laptop 3...
Scraping laptop 4...
Scraping laptop 5...
Scraping laptop 6...
Scraping laptop 7...
Scraping laptop 8...
Scraping laptop 9...


In [13]:
import pandas as pd 
pd.read_csv('sigma_laptops.csv')

,brand,model,cpu,gpu,ram,resolution_type,resolution,link,warranty,new_price,old_price,availability
0,Dell,3520,i5,UHD,8GB,FHD,1920 x 1080,https://www.sigma-computer.com/item?id=7652&na...,ضمــــــان سنــــــة,19199 LE,20000,Out Of Stock
1,HP,r1062ne,i5,RTX,16GB,FHD,Unknown,https://www.sigma-computer.com/item?id=7642&na...,ضمــــــــان سنـــــــــة,43999 LE,45000,In Stock
2,HP,r1066ne,i5,RTX,16GB,HD,1920x1080,https://www.sigma-computer.com/item?id=7641&na...,ضمــــــــان سنـــــــــــة,43499 LE,45000,In Stock
3,MSI,F13MG,i5,Iris,8GB,FHD,1920x1080,https://www.sigma-computer.com/item?id=7624&na...,ضمــــــان سنــــة,21749 LE,24500,In Stock
4,Lenovo,83DV0071PS,i7,RTX,16GB,FHD,Unknown,https://www.sigma-computer.com/item?id=7600&na...,ضمــــــان سنــــــة,46499 LE,47499,Out Of Stock
5,MSI,Vector 16 HX A14V,i9,RTX,32GB,QHD,2560x1600,https://www.sigma-computer.com/item?id=7583&na...,ضمـــــــان سنتـــــين وكــــــيل,133999 LE,138999,In Stock
6,MSI,RAIDER 18 HX A14V,i9,RTX,Unknown,UHD,3840x2400,https://www.sigma-computer.com/item?id=7582&na...,ضمـــــــان سنتـــــين وكــــــيل,176499 LE,180000,In Stock


In [3]:

import requests

import re

res = requests.get("https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch")

from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, 'html.parser')

all_laptops = soup.find_all('div', attrs= {'class': 'right-block'})


with open('sigma_laptops.csv', mode = 'w',  encoding='utf-8') as fd:
    fd.write('brand, model, cpu, gpu, ram, resolution_type, resolution, link, warranty, new_price, old_price, availability\n')
    


new_price = all_laptops[1].find('p').find('span', {'class': "price-new"}).text.strip()

old_price = all_laptops[1].find('p').find('span', {'class': "price-old"}).text.strip()

try:
    availability = all_laptops[1].find('div', {'class' : 'row'}).find('span', {'class': 'stock'}).text.strip()
except:
    availability = all_laptops[1].find('div', {'class' : 'row'}).find('span', {'class': 'stock_N'}).text.strip()

if float(old_price) > 10000:

    link1 = 'https://www.sigma-computer.com/' +  all_laptops[1].find('a').attrs['href']

    model_link = requests.get(link1)

    model_soup = BeautifulSoup(model_link.text, 'html.parser')

    try:

        warranty = model_soup.find('div', {'class': 'inner-box-desc'}).find('em').text

    except:

        pass


    model =  model_soup.find('div', {'class': 'model'}).find('a').text.strip()

    brand =  model_soup.find('div', {'class': 'brand'}).find('a').text.strip()


    try:
        if 'Processor' in model_soup.find('div', {'class': 'inner-box-desc'}).find('p').text:

            features = model_soup.find('div', {'class': 'inner-box-desc'}).find('p').get_text(separator="\n", strip=True)
            
            lines = features.split("\n")
        
            specs = {}

            keywords = ["Processor", "Graphics", "Memory", "Display", "GPU", "RAM"]

            current_key = None
            for line in lines:
                line = line.strip()
                if line in keywords:
                    current_key = line
                elif current_key:
                    specs[current_key] = line
                    current_key = None
            
            processor = specs['Processor']
            graphics = specs['Graphics']
            display = specs['Display']
            ram = specs['Memory']

            import re

            screen_resolution_pattern = r'\b(\d{3,4} x \d{3,4})\b'
            screen_resolution_match = re.search(screen_resolution_pattern, display)
            
            if screen_resolution_match:
                screen_resolution = screen_resolution_match.group(1)

            else:
                Screen_resolution=  'unknown'

            resolution_type_match = re.search(r'\b(FHD|QHD|4K|UHD|HD|WQHD|1080p)\b', display, re.IGNORECASE)
            resolution_type = resolution_type_match.group(1) if resolution_type_match else "Unknown"

            processor_match = re.search(r'\b(i[3579])\b', processor)
            processor = processor_match.group(1) if processor_match else "Unknown"

            gpu_match = re.search(r'(UHD|RTX|GTX|MX|Vega)\b', graphics, re.IGNORECASE)
            gpu = gpu_match.group(1) if gpu_match else "Unknown"
    
            ram_size_match = re.search(r'(\d+GB)', ram)
            ram = ram_size_match.group(1) if ram_size_match else "Unknown"
        

    except:

        if 'Processor' in model_soup.find('div', {'class': 'inner-box-desc'}).text:

            specs = model_soup.find('div', {'class': 'inner-box-desc'}).find_all('li')
  
            for i in range(len(specs)):
                if 'processor' in specs[i].text.lower():
                    processor = specs[i].text
                    processor_match = re.search(r'\b(i[3579])\b', processor)
                    if processor_match:
                        processor = processor_match.group(1)
                    else:
                        processor = "Unknown"

                elif 'ram' in specs[i].text.lower():

                    ram = specs[i].text
                    ram_match = re.search(r'(\d+GB)', ram)
                    ram = ram_match.group(1) if ram_match else "Unknown"
                    
                elif 'graphics' in specs[i].text.lower():

                    graphics = specs[i].text
                    gpu_match = re.search(r'(UHD|RTX|GTX|MX|Vega)\b', graphics, re.IGNORECASE)
                    gpu = gpu_match.group(1) if gpu_match else "Unknown"

                elif 'gpu' in specs[i].text.lower():

                    graphics = specs[i].text
                    gpu_match = re.search(r'(UHD|RTX|GTX|MX|Vega)\b', graphics, re.IGNORECASE)
                    gpu = gpu_match.group(1) if gpu_match else "Unknown"
            
                elif 'display' in specs[i].text.lower():

                    display = specs[i].text
                    screen_resolution_pattern = r'\b(\d{3,4} x \d{3,4})\b'
                    screen_resolution_match = re.search(screen_resolution_pattern, display)
                    
                    if screen_resolution_match:
                        screen_resolution = screen_resolution_match.group(1)

                    else:
                        Screen_resolution=  'unknown'

                    resolution_type_match = re.search(r'\b(FHD|QHD|4K|UHD|HD|WQHD|1080p)\b', display, re.IGNORECASE)
                    resolution_type = resolution_type_match.group(1) if resolution_type_match else "Unknown"
                    
                elif 'memory' in specs[i].text.lower():

                    ram = specs[i].text
                    ram_match = re.search(r'(\d+GB)', ram)
                    ram_match.group(1) if ram_match else "Unknown"
                                    

            


        




In [43]:
ram

'Unknown'